In [128]:
using DataFrames, Plots, LowRankModels, CSV

INFO: Precompiling module CSV.


In [201]:
Pkg.update("LowRankModels")

INFO: Updating METADATA...
INFO: Updating cache of BinDeps...
INFO: Updating cache of StatsBase...
INFO: Updating cache of FileIO...
INFO: Updating cache of Compat...
INFO: Updating cache of SpecialFunctions...
INFO: Updating LowRankModels master...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [330]:
train_data = readtable("train_data.csv")
test_data = readtable("test_data.csv")
delete!(train_data, :x )
delete!(test_data, :x )
#delete!(train_data, :Acquiror_CUSIP )
#delete!(test_data, :Acquiror_CUSIP )
#delete!(train_data, :Target_CUSIP )
#delete!(test_data, :Target_CUSIP )

,EV_to_EBIT,Acquiror_Cash_LTM,Target_Volatility_30,Target_Vol_jump_30,Target_Volatility_14,Target_Vol_jump_14,Target_Volatility_30_Shift,Target_Vol_jump_30_Shift,Target_Volatility_14_Shift,Target_Vol_jump_14_Shift,Acquiror_Volatility_30,Acquiror_Vol_jump_30,Acquiror_Volatility_14,Acquiror_Vol_jump_14,Acquiror_Volatility_30_Shift,Acquiror_Vol_jump_30_Shift,Acquiror_Volatility_14_Shift,Acquiror_Vol_jump_14_Shift,cap,cap_diff,Offer_EP,Past_EP,Offer_BP,Past_BP,Offer_Current_Price_Diff,price_premium,price_jump,rolling_price_premium,price_premium_diff,rolling_mean_long,rolling_mean_medium,rolling_mean_short,Democrat,First_Time,Experienced,Frequent,Investor,Investor_Group,Shareholders,nation_diff,industry_diff,fama_ind,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,year,new_Status
1,13.926,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,142.72,NA,0.031875,0.04788732394366197,0.675,1.0140845070422535,-0.034062499999999996,0.5033474178403755,0.4506713615023473,0.28822972222834425,0.21511769561203126,0.836,0.83,0.8333333333333334,-1,1,-1,-1,-1,-1,-1,1,1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,2008,1
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,39.1796,485.11039999999997,NA,NA,NA,NA,-0.08857722896609466,0.27513333333333334,0.16099999999999992,0.2882762555616776,-0.013142922228344234,0.84,0.83,0.8666666666666667,-1,1,-1,-1,-1,-1,-1,-1,1,11,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2008,1
3,68.395,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,53.71,14.889999999999993,0.04341463414634147,0.06778370144706779,0.7585365853658537,1.1843107387661842,-0.16534146341463418,0.5623099771515613,0.30259939070830144,0.28995920590799423,0.27235077124356705,0.84,0.84,0.9,-1,-1,1,-1,-1,-1,-1,1,1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,2008,1
4,24.491,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,55.0,75.41499999999999,0.049090909090909095,0.08925619834710745,0.5045454545454545,0.9173553719008264,-0.1671818181818181,0.8191818181818183,0.5133966942148761,0.2930042909042988,0.5261775272775195,0.844,0.84,0.9,-1,-1,1,-1,-1,-1,-1,1,1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,2008,1
5,7.264,152.66,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,187.11,713.928,0.08395061728395062,0.1175115207373272,0.8806584362139916,1.2327188940092166,-0.010522633744855958,0.4007695852534564,0.3846405529953917,0.29143402437510246,0.10933556087835394,0.844,0.84,0.9333333333333333,-1,1,-1,-1,-1,-1,-1,1,1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,2008,1
6,92.765,NA,0.47113299999999997,-0.018889888464069582,0.5787439999999999,0.5797720000000001,0.48168999999999995,0.02240768530330084,0.596009,0.5787439999999999,NA,NA,NA,NA,NA,NA,NA,NA,4143.492,182976.001,-0.28351955307262566,-0.39648437499999994,3.0586592178770946,4.27734375,-0.11492178770949724,0.3994375,0.237328125,0.2918340005504519,0.10760349944954811,0.844,0.84,0.9333333333333333,-1,-1,1,-1,-1,-1,-1,1,-1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,2008,1
7,29.225,1272.4,2.167646,0.027144644920940042,3.203329,3.2214490000000002,2.11128,-0.026003323420890734,3.020209,3.203329,0.22338400000000003,0.07039526960655151,0.29001,0.26042,0.24361999999999998,0.09058840382480371,0.31006999999999996,0.29001,480.96,5767.38,0.02333333333333333,0.03417412530512612,0.38333333333333336,0.5614320585842149,-0.011222222222222134,0.46560537021969084,0.4477046379170059,0.2922815071035618,0.17332386311612902,0.848,0.84,0.9333333333333333,-1,-1,1,-1,-1,-1,-1,1,1,13,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2008,1
8,16.604,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.6292949999999999,0.1380541344082813,0.715331,0.517215,0.616977,-0.01957428550

In [223]:
size(train_data)
#size(test_data)

(8326, 68)

In [331]:
train_x = train_data[:,1:end-47]
train_y = train_data[:,end]

test_x = test_data[:,1:end-47]
test_y = test_data[:,end]

2057-element DataArrays.DataArray{Int64,1}:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
 -1
  ⋮
  1
  1
  1
  1
  1
  1
  1
  1
  1
 -1
  1
  1

In [296]:
test_x

,EV_to_EBIT,Acquiror_Cash_LTM,Target_Volatility_30,Target_Vol_jump_30,Target_Volatility_14,Target_Vol_jump_14,Target_Volatility_30_Shift,Target_Vol_jump_30_Shift,Target_Volatility_14_Shift,Target_Vol_jump_14_Shift,Acquiror_Volatility_30,Acquiror_Vol_jump_30,Acquiror_Volatility_14,Acquiror_Vol_jump_14,Acquiror_Volatility_30_Shift,Acquiror_Vol_jump_30_Shift,Acquiror_Volatility_14_Shift,Acquiror_Vol_jump_14_Shift,cap,cap_diff,Offer_EP,Past_EP,Offer_BP,Past_BP,Offer_Current_Price_Diff,price_premium,price_jump
1,13.926,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,142.72,NA,0.031875,0.04788732394366197,0.675,1.0140845070422535,-0.034062499999999996,0.5033474178403755,0.4506713615023473
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,39.1796,485.11039999999997,NA,NA,NA,NA,-0.08857722896609466,0.27513333333333334,0.16099999999999992
3,68.395,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,53.71,14.889999999999993,0.04341463414634147,0.06778370144706779,0.7585365853658537,1.1843107387661842,-0.16534146341463418,0.5623099771515613,0.30259939070830144
4,24.491,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,55.0,75.41499999999999,0.049090909090909095,0.08925619834710745,0.5045454545454545,0.9173553719008264,-0.1671818181818181,0.8191818181818183,0.5133966942148761
5,7.264,152.66,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,187.11,713.928,0.08395061728395062,0.1175115207373272,0.8806584362139916,1.2327188940092166,-0.010522633744855958,0.4007695852534564,0.3846405529953917
6,92.765,NA,0.47113299999999997,-0.018889888464069582,0.5787439999999999,0.5797720000000001,0.48168999999999995,0.02240768530330084,0.596009,0.5787439999999999,NA,NA,NA,NA,NA,NA,NA,NA,4143.492,182976.001,-0.28351955307262566,-0.39648437499999994,3.0586592178770946,4.27734375,-0.11492178770949724,0.3994375,0.237328125
7,29.225,1272.4,2.167646,0.027144644920940042,3.203329,3.2214490000000002,2.11128,-0.026003323420890734,3.020209,3.203329,0.22338400000000003,0.07039526960655151,0.29001,0.26042,0.24361999999999998,0.09058840382480371,0.31006999999999996,0.29001,480.96,5767.38,0.02333333333333333,0.03417412530512612,0.38333333333333336,0.5614320585842149,-0.011222222222222134,0.46560537021969084,0.4477046379170059
8,16.604,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.6292949999999999,0.1380541344082813,0.715331,0.517215,0.616977,-0.01957428550997531,0.676564,0.715331,1268.01,NA,0.03005181347150259,0.04422079902409271,0.2134715025906736,0.3141201585849345,-0.06780829015544043,0.472485208905154,0.3712348276913694
9,NA,1043.68,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,131.99269999999999,11904.0423,-0.005921539600296078,-0.007619047619047619,0.007401924500370097,0.009523809523809525,0.0017401924500369814,0.2876666666666666,0.28861904761904766
10,7.357,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,74.385,NA,0.04827586206896552,0.06306306306306306,0.9379310344827586,1.2252252252252251,-0.06796551724137934,0.3073063063063063,0.21721621621621623


In [332]:
obs = observations(train_x)
loss = QuadLoss()
zero = ZeroReg()
k = 20
glrm = GLRM(train_x, loss, zero, zero, k, obs=obs);
equilibrate_variance!(glrm)
#add_offset!(glrm)

X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 91061.74063243499
Iteration 20: objective value = 76398.51261748662
Iteration 30: objective value = 66931.64139015162
Iteration 40: objective value = 61409.35313027658
Iteration 50: objective value = 56832.908516772004
Iteration 60: objective value = 53197.74483167691
Iteration 70: objective value = 51746.745974353245
Iteration 80: objective value = 49855.512885734104
Iteration 90: objective value = 48334.081077231494
Iteration 100: objective value = 45145.306996777625


([-0.603592 0.792346 … -0.015505 0.0943451; 1.0903 -0.405607 … -1.2124 0.183081; … ; 0.182508 -1.28817 … -0.218893 0.22171; 0.840769 0.0908058 … -0.728611 -0.695376], [-0.027428 0.129772 … 0.110299 0.0637375; 0.471615 1.32989 … 0.0923695 -0.0218234; … ; -1.75258 -2.48843 … -0.0366673 -0.024401; 1.38281 -2.03909 … -0.207311 -0.148761], LowRankModels.ConvergenceHistory("ProxGradGLRM", [8.67582e6, 4.02956e6, 3.55094e5, 2.02351e5, 1.58922e5, 129386.0, 1.12943e5, 104249.0, 98657.9, 94354.9  …  48116.6, 47864.9, 47593.3, 47344.0, 47126.2, 46799.5, 46453.4, 46106.4, 45610.4, 45145.3], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.48876, 2.01936, 2.50567, 2.96707, 3.44212, 3.90845, 4.4484, 5.02042, 5.59076  …  47.3597, 47.8426, 48.2997, 48.7756, 49.2876, 49.7692, 50.2301, 50.7037, 51.1652, 51.6329], [0.0, 0.0, 0.0

In [333]:
new_train_x = X'*Y

6981×27 Array{Float64,2}:
 -2.70717     0.000666252   1.60751    …  -0.239918    0.385311    0.0268523
 -3.07662    -0.177039     -0.122109      -0.0999331   0.393422    0.233415 
  0.136987    3.7742       -0.276925      -0.352522    0.519268    0.0131749
 -1.51898    -0.538495     -0.381696      -0.10334     0.266515    0.173968 
 -5.69743    -1.86851       2.10392       -0.394279    0.660958   -0.0406045
 -1.01466    -1.09764      -0.166185   …  -0.168237    0.444967    0.199452 
 -0.808506   -1.71903       1.20133       -0.148819    0.245473    0.039975 
  2.0286    -10.0798        2.9268        -0.271625    0.670604    0.184553 
 -0.617075   -1.89609      -0.327252      -0.062161    0.10462     0.0410744
  0.308043   -3.71523       0.139889      -0.0472378   0.286792    0.22697  
  2.45442     2.04799       0.249637   …  -0.197997    0.581706    0.28002  
 -0.108694    1.35591       1.90436       -0.122903    0.370396    0.173352 
  4.50407    13.9407       -0.255625      -0.07873

In [334]:
CSV.write("new_train_data.csv", DataFrame(new_train_x))

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd", IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "new_train_data.csv", 153, true, String["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10"  …  "x18", "x19", "x20", "x21", "x22", "x23", "x24", "x25", "x26", "x27"], false)

In [335]:
obs = observations(test_x)
loss = QuadLoss()
zero = ZeroReg()
k = 20
glrm = GLRM(test_x, loss, zero, zero, k, obs=obs);
equilibrate_variance!(glrm)
#add_offset!(glrm)

X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 25317.836574660076
Iteration 20: objective value = 18147.89293412464
Iteration 30: objective value = 16113.608952586277
Iteration 40: objective value = 15231.764981606415
Iteration 50: objective value = 14864.61729913843
Iteration 60: objective value = 14623.539650441684
Iteration 70: objective value = 14409.062815024396
Iteration 80: objective value = 14259.95949329473
Iteration 90: objective value = 14176.388155111237
Iteration 100: objective value = 14122.113896554856


([0.618605 -0.339484 … -0.626633 -0.444192; 2.04959 -0.00528992 … -0.00264917 0.725944; … ; -1.63419 -0.89584 … -1.42192 -0.0757362; -2.2652 -0.305907 … -1.14221 -0.194849], [1.59119 -1.76102 … -0.138412 -0.223799; 1.40233 -0.411651 … 0.267416 0.195735; … ; 2.16131 -0.464803 … -0.042895 0.0154816; -0.155153 -0.593954 … 0.219518 0.13451], LowRankModels.ConvergenceHistory("ProxGradGLRM", [2.65523e6, 1.10476e6, 2.89666e5, 80429.9, 52811.5, 42123.9, 34995.4, 31639.5, 28846.4, 26811.5  …  14169.9, 14164.0, 14158.1, 14152.2, 14146.4, 14141.4, 14136.7, 14131.3, 14126.7, 14122.1], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.332561, 0.488964, 0.635866, 0.779826, 0.930896, 1.12181, 1.28043, 1.43056, 1.58403  …  14.022, 14.167, 14.3201, 14.4676, 14.6138, 14.7609, 14.907, 15.0523, 15.1973, 15.3471], [0.0, 0.0, 0.0, 

In [336]:
new_test_x = X'*Y

2057×27 Array{Float64,2}:
  -1.34339     1.48922    -2.07592    -2.52878    …   0.494089    0.469964 
  -4.16925     2.99361    -1.5932      2.47624        0.270693    0.16333  
  -0.904715   -0.463634    0.42055     1.85878        0.555801    0.316441 
  -3.90122     0.0626407  -0.747951    1.28932        0.801208    0.539752 
  10.5838     -0.320629    2.37897     2.85293        0.39295     0.399301 
   8.63454     1.25689     0.463253   -0.0178726  …   0.357255    0.283874 
   0.036212   -2.19472     2.33248    -0.390837       0.52296     0.469663 
   0.0780826   2.57257     0.29285     1.62858        0.456449    0.364726 
  -3.84107    -1.24141    -1.87793    -0.0218594      0.27957     0.303302 
  -4.51638     1.44687     1.1091     -0.171528       0.301647    0.226808 
   2.75472    -3.59221    -0.627712   -3.14593    …   0.310315    0.307073 
  -1.30239     5.33815     2.64566     3.62277        0.375579    0.313642 
   3.0877     -2.38088     1.82116     0.566726       0.761144

In [337]:
CSV.write("new_test_data.csv", DataFrame(new_test_x))

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd", IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "new_test_data.csv", 153, true, String["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10"  …  "x18", "x19", "x20", "x21", "x22", "x23", "x24", "x25", "x26", "x27"], false)